In [64]:
### Necessary Library

import os
import torch
import math

In [65]:
#######################################################################################
################################# LSTM MODEL ##########################################
#######################################################################################

import torch.nn as nn


## Creating a class of nn.Module to include RNN with LSTM Units
class LSTM_Model(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, hidden_size, batch_size = 1, n_layers = 1):
        
        super(LSTM_Model, self).__init__()
        
        ## Creating an embedding object to create 'embedding_size' dimensional encoding
        self.encoder = nn.Embedding(vocab_size,embedding_size)
        
        self.lstm = nn.LSTM(embedding_size,hidden_size,n_layers)
        
        ## The output size is assigned as hidden_size * vocab_size 
        ## (Since the output is one word from the entire vocab)
        self.linear_output = nn.Linear(hidden_size,vocab_size)
        
        self.init_weights()
        
        ## More variables
        self.hidden_size = hidden_size
        self.n_layers = n_layers
    
        
    def init_weights(self):
        random_range = 0.1
        
        ## Setting random values for different layers
        self.encoder.weight.data.uniform_(-random_range, random_range)
        
        self.linear_output.bias.data.zero_()
        self.linear_output.weight.data.uniform_(-random_range, random_range)

    # Defining the forward layer    
    def forward(self, input, hidden):
        
        #Passing through the embedding layer
        embedding_input = self.encoder(input)
        
        lstm_out, hidden = self.lstm(embedding_input, hidden)
        
        decoded = self.linear_output(lstm_out.view(lstm_out.size(0)*lstm_out.size(1), lstm_out.size(2)))
        
        return decoded.view(lstm_out.size(0), lstm_out.size(1), decoded.size(1)), hidden

    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters())
        
        return (weight.new_zeros(self.n_layers, batch_size, self.hidden_size), \
                    weight.new_zeros(self.n_layers, batch_size, self.hidden_size))
    
######################################################################################
######################################################################################

In [66]:
################################## FUNCTIONS TO READ THE DATA ########################

# Dictionary class to create word2idx and idx2word objects 
#(Mapping for a word to an id and vice versa)

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)
    

    
#### Tokenize function to convert text file to torch idx file #####################

def tokenize(path):
    """Tokenizes a text file."""
    assert os.path.exists(path)
    # Add words to the dictionary
    
    with open(path, 'r', encoding="utf8") as f:
        
        
        tokens = 0
        for line in f:
            words = line.split()
            tokens += len(words)
            for word in words:
                dictionary.add_word(word)

    # Tokenize file content
    with open(path, 'r', encoding="utf8") as f:
        ids = torch.LongTensor(tokens)
        token = 0
        for line in f:
            words = line.split()
            for word in words:
                ids[token] = dictionary.word2idx[word]
                token += 1

    return ids

######################################################################################
######################################################################################

In [126]:
########################## Additional Functions for training the model ##############

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)


#### Separate data into batch sizes

def batch_split(data, pos, bs = mini_batch_size):
    
    nbatch = data.size(0) // mini_batch_size
    
    data = data.narrow(0, 0, nbatch * mini_batch_size)

    # Evenly divide the data across the mini_batch_size batches.
    data = data.view(mini_batch_size, -1).t().contiguous().to(device)
    
    
    in_data = data[pos:pos+bs]
    out_data = data[(pos+1):(pos+bs+1)].view(-1)
    
    return in_data,out_data
    
    
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [127]:
# def batchify(data, bsz):
#     # Work out how cleanly we can divide the dataset into bsz parts.
#     nbatch = data.size(0) // bsz
#     # Trim off any extra elements that wouldn't cleanly fit (remainders).
#     data = data.narrow(0, 0, nbatch * bsz)
#     # Evenly divide the data across the bsz batches.
#     data = data.view(bsz, -1).t().contiguous()
#     return data.to(device)

In [185]:
def train():
    
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    #start_time = time.time()
    
    hidden = model.init_hidden(mini_batch_size)
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, mini_batch_size)):
        
        
        data, targets = batch_split(train_data, i,mini_batch_size)
       
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        
        model.zero_grad()
        
        ## Calling the forward pass of the model 
        output, hidden = model(data, hidden)
        
        ## Calculating the loss
        loss = criterion(output.view(-1, vocab_size), targets)
        
        ## Computing gradient and optimizing
        loss.backward()
        optimizer.step()
        

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
        
        
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            #elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // mini_batch_size, lr,\
                    #elapsed * 1000 / args.log_interval, 
                        cur_loss, math.exp(cur_loss)))
            total_loss = 0
            #start_time = time.time()

In [202]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    
    hidden = model.init_hidden(eval_batch_size)
    
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, eval_batch_size):
            
            data, targets = batch_split(data_source, i,eval_batch_size)
            
            output, hidden = model(data, hidden)
            
            output_flat = output.view(-1, vocab_size)
                      
            total_loss +=  criterion(output_flat, targets).item()
            
            hidden = repackage_hidden(hidden)
            
    return total_loss / (len(data_source) - 1)

#### Reading the data

In [146]:
#Initializing dictionary object
dictionary = Dictionary()

train_data = tokenize('trn-wiki.txt')
val_data = tokenize('dev-wiki.txt')

In [192]:
train_data = train_data[0:320]
#val_data = val_data[0:320]
val_data = train_data[0:320]

### Input Arguments

In [204]:
#vocab_size = len(dictionary)

vocab_size = len(set(train_data))

embedding_size = 32 
hidden_size = 32
mini_batch_size = 1
eval_batch_size = mini_batch_size
n_layers = 1 # For single layer LSTM

##### Creating the model object from the LSTM_Model in the models.py

model = LSTM_Model(vocab_size, embedding_size, hidden_size) ## LSTM Model

optimizer = torch.optim.SGD(model.parameters(), lr=0.1) ## SGD Optimizer
criterion = nn.CrossEntropyLoss() ## Crossentropy Loss function


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#### Model parameters 

lr = 10 # Learning rate
epochs = 20 #
gradient_clip_val = 1
log_interval = 1

ntokens = vocab_size

In [205]:
### Running the data over different epochs
for epoch in range(1, epochs+1):
    #epoch_start_time = time.time()
    train()
    val_loss = evaluate(val_data)
    print(val_loss)

| epoch   1 |     1/  320 batches | lr 10.00 | loss 16.04 | ppl 9289838.13
| epoch   1 |     2/  320 batches | lr 10.00 | loss  6.81 | ppl   904.19
| epoch   1 |     3/  320 batches | lr 10.00 | loss 11.49 | ppl 97697.35
| epoch   1 |     4/  320 batches | lr 10.00 | loss  6.56 | ppl   707.29
| epoch   1 |     5/  320 batches | lr 10.00 | loss  9.67 | ppl 15766.12
| epoch   1 |     6/  320 batches | lr 10.00 | loss  6.52 | ppl   676.76
| epoch   1 |     7/  320 batches | lr 10.00 | loss  9.87 | ppl 19259.15
| epoch   1 |     8/  320 batches | lr 10.00 | loss  6.50 | ppl   664.65
| epoch   1 |     9/  320 batches | lr 10.00 | loss  7.40 | ppl  1628.46
| epoch   1 |    10/  320 batches | lr 10.00 | loss  9.19 | ppl  9813.31
| epoch   1 |    11/  320 batches | lr 10.00 | loss  7.19 | ppl  1329.14
| epoch   1 |    12/  320 batches | lr 10.00 | loss  8.40 | ppl  4433.70
| epoch   1 |    13/  320 batches | lr 10.00 | loss  6.78 | ppl   876.06
| epoch   1 |    14/  320 batches | lr 10.00 | lo

| epoch   1 |   189/  320 batches | lr 10.00 | loss  9.12 | ppl  9167.87
| epoch   1 |   190/  320 batches | lr 10.00 | loss  6.85 | ppl   948.60
| epoch   1 |   191/  320 batches | lr 10.00 | loss  8.91 | ppl  7426.31
| epoch   1 |   192/  320 batches | lr 10.00 | loss  7.87 | ppl  2605.73
| epoch   1 |   193/  320 batches | lr 10.00 | loss  7.46 | ppl  1737.99
| epoch   1 |   194/  320 batches | lr 10.00 | loss  7.80 | ppl  2441.50
| epoch   1 |   195/  320 batches | lr 10.00 | loss  6.64 | ppl   767.89
| epoch   1 |   196/  320 batches | lr 10.00 | loss  8.15 | ppl  3475.69
| epoch   1 |   197/  320 batches | lr 10.00 | loss  5.97 | ppl   391.30
| epoch   1 |   198/  320 batches | lr 10.00 | loss  7.86 | ppl  2598.81
| epoch   1 |   199/  320 batches | lr 10.00 | loss  7.12 | ppl  1241.33
| epoch   1 |   200/  320 batches | lr 10.00 | loss  8.10 | ppl  3284.95
| epoch   1 |   201/  320 batches | lr 10.00 | loss 12.37 | ppl 235568.20
| epoch   1 |   202/  320 batches | lr 10.00 | los

6.842583348384845
| epoch   2 |     1/  320 batches | lr 10.00 | loss 15.89 | ppl 7976168.59
| epoch   2 |     2/  320 batches | lr 10.00 | loss  7.97 | ppl  2893.18
| epoch   2 |     3/  320 batches | lr 10.00 | loss  8.75 | ppl  6305.84
| epoch   2 |     4/  320 batches | lr 10.00 | loss  6.17 | ppl   475.85
| epoch   2 |     5/  320 batches | lr 10.00 | loss  4.16 | ppl    64.10
| epoch   2 |     6/  320 batches | lr 10.00 | loss 11.16 | ppl 69990.89
| epoch   2 |     7/  320 batches | lr 10.00 | loss  8.69 | ppl  5926.05
| epoch   2 |     8/  320 batches | lr 10.00 | loss  7.58 | ppl  1952.11
| epoch   2 |     9/  320 batches | lr 10.00 | loss  6.60 | ppl   736.67
| epoch   2 |    10/  320 batches | lr 10.00 | loss  8.29 | ppl  3984.39
| epoch   2 |    11/  320 batches | lr 10.00 | loss 17.70 | ppl 48625070.55
| epoch   2 |    12/  320 batches | lr 10.00 | loss  7.35 | ppl  1551.08
| epoch   2 |    13/  320 batches | lr 10.00 | loss  6.71 | ppl   817.84
| epoch   2 |    14/  320 ba

| epoch   2 |   186/  320 batches | lr 10.00 | loss  7.31 | ppl  1492.47
| epoch   2 |   187/  320 batches | lr 10.00 | loss  8.63 | ppl  5577.62
| epoch   2 |   188/  320 batches | lr 10.00 | loss 11.54 | ppl 102996.76
| epoch   2 |   189/  320 batches | lr 10.00 | loss 10.22 | ppl 27476.95
| epoch   2 |   190/  320 batches | lr 10.00 | loss 19.64 | ppl 338958398.48
| epoch   2 |   191/  320 batches | lr 10.00 | loss 12.51 | ppl 272034.27
| epoch   2 |   192/  320 batches | lr 10.00 | loss  7.57 | ppl  1936.57
| epoch   2 |   193/  320 batches | lr 10.00 | loss  7.44 | ppl  1700.90
| epoch   2 |   194/  320 batches | lr 10.00 | loss 12.25 | ppl 209114.86
| epoch   2 |   195/  320 batches | lr 10.00 | loss  6.59 | ppl   727.85
| epoch   2 |   196/  320 batches | lr 10.00 | loss  8.36 | ppl  4278.46
| epoch   2 |   197/  320 batches | lr 10.00 | loss 10.46 | ppl 34731.73
| epoch   2 |   198/  320 batches | lr 10.00 | loss  7.97 | ppl  2902.60
| epoch   2 |   199/  320 batches | lr 10.00

7.409538397594679
| epoch   3 |     1/  320 batches | lr 10.00 | loss 14.39 | ppl 1768878.26
| epoch   3 |     2/  320 batches | lr 10.00 | loss  9.77 | ppl 17470.76
| epoch   3 |     3/  320 batches | lr 10.00 | loss  4.98 | ppl   145.33
| epoch   3 |     4/  320 batches | lr 10.00 | loss 13.25 | ppl 568587.11
| epoch   3 |     5/  320 batches | lr 10.00 | loss 14.93 | ppl 3045536.93
| epoch   3 |     6/  320 batches | lr 10.00 | loss  5.38 | ppl   217.20
| epoch   3 |     7/  320 batches | lr 10.00 | loss  8.00 | ppl  2981.27
| epoch   3 |     8/  320 batches | lr 10.00 | loss  6.92 | ppl  1011.84
| epoch   3 |     9/  320 batches | lr 10.00 | loss  6.88 | ppl   969.09
| epoch   3 |    10/  320 batches | lr 10.00 | loss 10.56 | ppl 38576.00
| epoch   3 |    11/  320 batches | lr 10.00 | loss  9.91 | ppl 20117.58
| epoch   3 |    12/  320 batches | lr 10.00 | loss  9.69 | ppl 16091.30
| epoch   3 |    13/  320 batches | lr 10.00 | loss 12.08 | ppl 175605.36
| epoch   3 |    14/  320 b

...

7.549211061112933
| epoch  19 |     1/  320 batches | lr 10.00 | loss 28.52 | ppl 2434996730730.02
| epoch  19 |     2/  320 batches | lr 10.00 | loss 11.04 | ppl 62156.68
| epoch  19 |     3/  320 batches | lr 10.00 | loss  0.04 | ppl     1.04
| epoch  19 |     4/  320 batches | lr 10.00 | loss 17.03 | ppl 24853262.84
| epoch  19 |     5/  320 batches | lr 10.00 | loss  0.00 | ppl     1.00
| epoch  19 |     6/  320 batches | lr 10.00 | loss 18.90 | ppl 160826980.72
| epoch  19 |     7/  320 batches | lr 10.00 | loss  8.89 | ppl  7275.62
| epoch  19 |     8/  320 batches | lr 10.00 | loss  5.83 | ppl   341.87
| epoch  19 |     9/  320 batches | lr 10.00 | loss  0.10 | ppl     1.11
| epoch  19 |    10/  320 batches | lr 10.00 | loss 36.66 | ppl 8369704113731189.00
| epoch  19 |    11/  320 batches | lr 10.00 | loss 20.69 | ppl 965057617.00
| epoch  19 |    12/  320 batches | lr 10.00 | loss  4.47 | ppl    87.76
| epoch  19 |    13/  320 batches | lr 10.00 | loss  8.92 | ppl  7481.38
| e

| epoch  19 |   182/  320 batches | lr 10.00 | loss 11.43 | ppl 91608.87
| epoch  19 |   183/  320 batches | lr 10.00 | loss  0.00 | ppl     1.00
| epoch  19 |   184/  320 batches | lr 10.00 | loss  6.71 | ppl   821.73
| epoch  19 |   185/  320 batches | lr 10.00 | loss 10.23 | ppl 27759.83
| epoch  19 |   186/  320 batches | lr 10.00 | loss  8.19 | ppl  3596.00
| epoch  19 |   187/  320 batches | lr 10.00 | loss  0.64 | ppl     1.90
| epoch  19 |   188/  320 batches | lr 10.00 | loss  0.00 | ppl     1.00
| epoch  19 |   189/  320 batches | lr 10.00 | loss  0.00 | ppl     1.00
| epoch  19 |   190/  320 batches | lr 10.00 | loss  5.77 | ppl   320.90
| epoch  19 |   191/  320 batches | lr 10.00 | loss 15.63 | ppl 6111395.28
| epoch  19 |   192/  320 batches | lr 10.00 | loss  7.21 | ppl  1351.93
| epoch  19 |   193/  320 batches | lr 10.00 | loss  7.17 | ppl  1296.19
| epoch  19 |   194/  320 batches | lr 10.00 | loss  9.73 | ppl 16862.21
| epoch  19 |   195/  320 batches | lr 10.00 | lo

7.121390764242429
| epoch  20 |     1/  320 batches | lr 10.00 | loss  7.86 | ppl  2592.60
| epoch  20 |     2/  320 batches | lr 10.00 | loss  7.30 | ppl  1481.57
| epoch  20 |     3/  320 batches | lr 10.00 | loss  8.06 | ppl  3176.28
| epoch  20 |     4/  320 batches | lr 10.00 | loss  7.36 | ppl  1569.41
| epoch  20 |     5/  320 batches | lr 10.00 | loss  6.71 | ppl   822.30
| epoch  20 |     6/  320 batches | lr 10.00 | loss 14.55 | ppl 2079799.27
| epoch  20 |     7/  320 batches | lr 10.00 | loss 12.08 | ppl 176879.72
| epoch  20 |     8/  320 batches | lr 10.00 | loss  5.62 | ppl   276.49
| epoch  20 |     9/  320 batches | lr 10.00 | loss  2.76 | ppl    15.84
| epoch  20 |    10/  320 batches | lr 10.00 | loss 10.79 | ppl 48742.83
| epoch  20 |    11/  320 batches | lr 10.00 | loss 12.32 | ppl 224511.02
| epoch  20 |    12/  320 batches | lr 10.00 | loss  8.92 | ppl  7487.94
| epoch  20 |    13/  320 batches | lr 10.00 | loss  6.09 | ppl   442.83
| epoch  20 |    14/  320 bat

| epoch  20 |   182/  320 batches | lr 10.00 | loss  7.87 | ppl  2605.55
| epoch  20 |   183/  320 batches | lr 10.00 | loss  0.00 | ppl     1.00
| epoch  20 |   184/  320 batches | lr 10.00 | loss  9.88 | ppl 19462.77
| epoch  20 |   185/  320 batches | lr 10.00 | loss 10.94 | ppl 56249.45
| epoch  20 |   186/  320 batches | lr 10.00 | loss  4.84 | ppl   126.61
| epoch  20 |   187/  320 batches | lr 10.00 | loss  2.92 | ppl    18.59
| epoch  20 |   188/  320 batches | lr 10.00 | loss  0.01 | ppl     1.01
| epoch  20 |   189/  320 batches | lr 10.00 | loss  0.02 | ppl     1.02
| epoch  20 |   190/  320 batches | lr 10.00 | loss  5.33 | ppl   206.12
| epoch  20 |   191/  320 batches | lr 10.00 | loss 14.11 | ppl 1340525.16
| epoch  20 |   192/  320 batches | lr 10.00 | loss 11.39 | ppl 88224.17
| epoch  20 |   193/  320 batches | lr 10.00 | loss 11.43 | ppl 92169.19
| epoch  20 |   194/  320 batches | lr 10.00 | loss  0.23 | ppl     1.26
| epoch  20 |   195/  320 batches | lr 10.00 | lo

7.78661042395804


In [167]:
len(val_data)

320

In [206]:
#### Calculate perplexity
#### Batch inputs such a way that only same sentences are together